In [1]:
import pandas as pd

df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

not_fit = True


In [8]:
#print(df['Page content'][2])

#content = procd.parse_html(df['Page content'][2])


<html><head><div class="article-info"><span class="byline "><a href="/author/sam-laird/"><img alt="2016%2f09%2f15%2f63%2fhttpsd2mhye01h4nj2n.cloudfront.netmediazgkymde1lza2.9814b" class="author_image" src="http://i.amz.mshcdn.com/-qaMPB8aiQeIaoBhqlU0OLjA07A=/90x90/2016%2F09%2F15%2F63%2Fhttpsd2mhye01h4nj2n.cloudfront.netmediaZgkyMDE1LzA2.9814b.jpg"/></a><span class="author_name">By <a href="/author/sam-laird/">Sam Laird</a></span><time datetime="Wed, 07 May 2014 19:15:20 +0000">2014-05-07 19:15:20 UTC</time></span></div></head><body><h1 class="title">Ballin': 2014 NFL Draft Picks Get to Choose Their Own Walk-Out Music</h1><figure class="article-image"><img alt="This Dec. 8, 2012 file photo shows Texas A&amp;M quarterback Johnny Manziel gesturing as he talks to reporters after winning the Heisman Trophy in New York." class="microcontent" data-fragment="lead-image" data-image="http://i.amz.mshcdn.com/CADSAjPf_UHvl5Z9OqlkPWABIGU=/950x534/2014%2F05%2F07%2Fff%2FJohnnyOK.cae2a.jpg" data-micro

In [ ]:
text = BeautifulSoup(df['Page content'][2])

In [8]:
import numpy as np
from bs4 import BeautifulSoup
from collections import defaultdict
import src.proc_data as procd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
"""
train_parse = np.zeros((df.values.shape[0], 5),dtype=object)

for i in range(df['Page content'].values.shape[0]):
    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3]\
            = procd.parse_html(df['Page content'][i])
    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))

pd.DataFrame(data=train_parse, columns=['title', 'content', 'author', 'n_fig', 'n_words']).to_csv('train_parsed.csv')

train_parse = np.zeros((df_test.values.shape[0], 5),dtype=object)

for i in range(df_test['Page content'].values.shape[0]):
    train_parse[i, 0], train_parse[i, 1], train_parse[i, 2],  train_parse[i, 3] \
            = procd.parse_html(df_test['Page content'][i])
    train_parse[i, 4] = len(tokenizer_stem_nostop(train_parse[i, 1]))

pd.DataFrame(data=train_parse, columns=['title', 'content', 'author','n_fig', 'n_words']).to_csv('test_parsed.csv')
"""

In [3]:
df = pd.read_csv('train_parsed.csv')
df_test = pd.read_csv('test_parsed.csv')


In [3]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
nltk.download('stopwords')
stop = stopwords.words('english')

def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]

[nltk_data] Downloading package stopwords to /Users/AtSu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
p_data = df[df['Popularity'] > 0] 
n_data = df[df['Popularity'] < 0]

tokenizer_stem_nostop('amazon microsoft women hahahaha')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [14]:
import itertools, operator

def extract_term(top, tfidf):
    feat = tfidf.get_feature_names()
    idf = tfidf.idf_
    idx = idf.argsort()
    sorted_score = []
    
    for i in idx: sorted_score.append(idf[i])
    sorted_smallest = list(zip(feat, sorted_score))
    sorted_smallest = sorted(sorted_smallest, key=operator.itemgetter(1))
    sorted_highest = sorted_smallest[::-1]
    term = []
    print('[vocabularies with smallest idf scores]')
    for i in range(top):
        #print('%s : %f' %(sorted_smallest[i][0], sorted_smallest[i][1]))
        term.append(sorted_smallest[i][0])
        
    print('[vocabularies with height idf scores]')
    for i in range(top):
        #print('%s : %f' %(sorted_highest[i][0], sorted_highest[i][1]))
        term.append(sorted_highest[i][0])
    return term

In [15]:
import scipy as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
import _pickle as pkl

stop = stopwords.words('english')

p = p_data['Page content']
n = n_data['Page content']

if not_fit:
    tfidf = TfidfVectorizer(ngram_range=(1,1),
                        #preprocessor=procd.parse_html,
                        tokenizer=tokenizer_stem_nostop,
                        max_features=2**10,
                        max_df=0.6,
                        min_df=0.05,
                        sublinear_tf=True
                        )
    not_fit = False
    tfidf_p = tfidf.fit(p)
    pkl.dump(tfidf_p, open('tfidf_p.pkl', 'wb'))
    tfidf_n = tfidf.fit(n)
    pkl.dump(tfidf_n, open('tfidf_n.pkl', 'wb'))

tfidf_p = pkl.load(open('tfidf_p.pkl', 'rb'))
tfidf_n = pkl.load(open('tfidf_n.pkl', 'rb'))

t_p = extract_term(40, tfidf_p)
t_n = extract_term(40, tfidf_n)
   
extra_stop = []
for t in t_p:
    if t in t_n: extra_stop.append(t)
stop += extra_stop

print(stop)

[vocabularies with smallest idf scores]
[vocabularies with height idf scores]
[vocabularies with smallest idf scores]
[vocabularies with height idf scores]
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'm

In [166]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from time import time

t0 = time()
tfidf = TfidfVectorizer(ngram_range=(2,4),
                        #preprocessor=procd.parse_html,
                        tokenizer=tokenizer_stem_nostop,
                        max_features=2**9,
                        sublinear_tf=True
                        )
X_train = tfidf.fit_transform(df['Page content'])
duration = time() - t0

print('Finishing fitting TFIDF : %f s' %(duration))
print('dimension: %d' %(X_train.shape[1]))
#sc_x = MaxAbsScaler()
#X_train = sc_x.fit_transform(X_train)
y_train = df['Popularity']


Finishing fitting TFIDF : 328.600646 s
dimension: 512


In [167]:
print(tfidf.get_feature_names())

['accord new', 'accord report', 'across countri', 'across u', 'activ user', 'addit report', 'afp getti', 'afp getti imag', 'air forc', 'airlin flight', 'allow user', 'also help', 'also includ', 'also known', 'also offer', 'also publish', 'also said', 'also use', 'also work', 'android app', 'android devic', 'android wear', 'announc new', 'answer question', 'app also', 'app avail', 'app like', 'app store', 'app use', 'appl pay', 'appl tv', 'appl watch', 'around world', 'articl origin', 'articl origin publish', 'articl origin publish space', 'ask question', 'associ press', 'barack obama', 'batteri life', 'behind scene', 'best way', 'big data', 'black friday', 'black white', 'blog post', 'brand new', 'break bad', 'busi owner', 'carbon dioxid', 'career opportun', 'chang way', 'check video', 'chief execut', 'climat chang', 'co founder', 'come back', 'comic book', 'comic con', 'comment homepag', 'comment homepag imag', 'comment imag', 'comment imag courtesi', 'comment imag via', 'compani also

In [124]:
from sklearn.feature_extraction.text import HashingVectorizer

t0 = time()
hashvec = HashingVectorizer(ngram_range=(3,4),
                           tokenizer=tokenizer_stem_nostop,
                           n_features=2**9)
X_train = hashvec.transform(df['Page content'])
duration = time() - t0
print('Finishing fitting HASH : %f s' %(duration))
print('dimension: %d' %(X_train.shape[1]))

Finishing fitting HASH : 149.701819 s
dimension: 512


In [190]:
from scipy.sparse import hstack
from sklearn.preprocessing import PolynomialFeatures
train_weekday = pd.read_csv('train_weekday.csv').values
train_time = pd.read_csv('train_time.csv').values
#poly = PolynomialFeatures(degree=3)
#train_weekday = poly.fit_transform(train_weekday)
#train_time = poly.fit_transform(train_time)

#X_train_aug = hstack([train_weekday, train_time])

X_train_aug = np.concatenate((train_weekday, train_time), axis=1)
#X_train_aug = train_weekday * train_time
print(X_train_aug.shape)

(27643, 1)
(27643, 7)
(27643, 7)


In [24]:
def get_stream(path, size):
    for chunk in pd.read_csv(path, chunksize=size):
        yield chunk
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [191]:
from sklearn.model_selection import train_test_split
# hold out testing set
y_train = df['Popularity']
X_htrain, X_htest, y_htrain, y_htest = train_test_split(X_train_aug, y_train, test_size=0.3, random_state=0)


In [192]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.svm import LinearSVC



SGD = SGDClassifier(alpha=0.001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=50, n_jobs=1,
       penalty='elasticnet', power_t=0.5, random_state=0, shuffle=True,
       verbose=0, warm_start=False)
SGD.fit(X_htrain, y_htrain)
pred = SGD.predict(X_htrain)
pred_t = SGD.predict(X_htest)

score_train = metrics.accuracy_score(y_htrain, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)

print("SGD score (train): %f, (test): %f"%(score_train, score_test))
ada = AdaBoostClassifier(base_estimator=SGD, algorithm='SAMME', n_estimators=10)
ada = ada.fit(X_htrain, y_htrain)
pred = ada.predict(X_htrain)
pred_t = ada.predict(X_htest)

score_train = metrics.accuracy_score(y_htrain, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)

print("SGD score (train): %f, (test): %f"%(score_train, score_test))

SGD.fit(X_train_aug,y_train)
pred = SGD.predict(X_htrain)
pred_t = SGD.predict(X_htest)
score_train = metrics.accuracy_score(y_htrain, pred)
score_test = metrics.accuracy_score(y_htest, pred_t)
print("SGD score (train): %f, (test): %f"%(score_train, score_test))
candidates = []
candidates.append({'clf':SGD, 'C':1, 'score':score})


SGD score (train): 0.542067, (test): 0.547932
SGD score (train): 0.505581, (test): 0.509828
SGD score (train): 0.540879, (test): 0.546485


In [54]:
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.linear_model import LogisticRegression

y_train = df['Popularity']
batch_size = 2000

classes = np.array([-1, 1])
iters = int((27643+batch_size-1)/(batch_size))
Cs = [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 3.0, 5.0, 7.0, 10.0]

best_score = 0
candidates = []
"""
t0 = time()
clf = SVC(max_iter=15000, random_state=0)
clf.fit(X_train, df['Popularity'])
candidates.append({'clf':clf, 'C':C, 'score':score})
score = clf.score(X_train, df['Popularity'])
print(score)
candidates.append({'clf':clf, 'C':0.001, 'score':score})
print('training duration : %f' %(time()-t0))
"""

selector = SelectFromModel(estimator=LinearSVC(dual=False, tol=1e-1))
X_train_sel = selector.fit_transform(X_train, y_train)
#X_train_sel = X_train
#poly = PolynomialFeatures(degree=3)
#X_train_poly = poly.fit_transform(X_train_sel.toarray())
X_train_poly = X_train_sel
#sc_x = StandardScaler(with_mean=False)
#X_train_sel = sc_x.fit_transform(X_train_sel)
print(X_train_sel.shape)
clf = MultinomialNB(alpha=0.01)
clf.fit(abs(X_train_sel), y_train)
pred = clf.predict(X_train_sel)
score = metrics.accuracy_score(y_train, pred)
print("MNB score : %f" %(score))
candidates.append({'clf':clf, 'C':0, 'score':score})

SGD = SGDClassifier(alpha=0.001, loss='hinge', penalty='l2')
SGD.fit(X_train_sel, y_train)
pred = SGD.predict(X_train_sel)
score = metrics.accuracy_score(y_train, pred)
print("SGD score : %f" %(score))
candidates.append({'clf':SGD, 'C':1, 'score':score})


ELA = SGDClassifier(alpha=0.001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=50, n_jobs=1,
       penalty='elasticnet', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)
ELA.fit(X_train_sel, y_train)
pred = ELA.predict(X_train_sel)
score = metrics.accuracy_score(y_train, pred)
print("SGD score : %f" %(score))
candidates.append({'clf':ELA, 'C':2, 'score':score})

sv = LinearSVC(C=3.0, tol=0.00001, max_iter=10000)
sv.fit(X_train_sel, y_train)
pred = sv.predict(X_train_sel)

score = metrics.accuracy_score(y_train, pred)
print("svc score : %f" %(score))
candidates.append({'clf':sv, 'C':3, 'score':score})

lr = LogisticRegression(C=0.1)
lr.fit(X_train_sel, y_train)
pred = lr.predict(X_train_sel)
score = metrics.accuracy_score(y_train, pred)
print("lr score : %f" %(score))
candidates.append({'clf':sv, 'C':4, 'score':score})

"""
for C in Cs:
    clf = Pipeline([
            ('feature_selection', SelectFromModel(estimator=LinearSVC(penalty='l1', dual=False, tol=1e-3))),
            ('classification', LinearSVC(C=C))
        ])
    #clf = LinearSVC(C=C, dual=False, tol=0.00001, max_iter=2000)
    clf.fit(X_train_sel, y_train)
    pred = clf.predict(X_train_sel)

    score = metrics.accuracy_score(y_train, pred)
    print("C(%f):%f"%(C,score))
    candidates.append({'clf':clf, 'C':C, 'score':score})
"""
"""
for C in Cs:
    alpha = C / 1000.0
    clf = SGDClassifier(loss='hinge', alpha=alpha)
    #stream = get_stream(path='train.csv', size=batch_size)
    for i in range(iters):
        #batch = next(stream)
        #X_train, y_train = batch['Page content'], batch['Popularity']
        start = (i)*batch_size
        end = start + batch_size
        if end >= X_train.shape[0] : end = X_train.shape[0]
        
        batch_X = X_train[start:end]
        batch_y = y_train[start:end]
        if X_train is None:
            break
        #X_train = sc_x.transform(tfidf.transform(X_train))
        clf.partial_fit(batch_X, batch_y, classes=classes)
    
        score = clf.score(batch_X, batch_y)
        print('[{}/{}] {}'.format((i+1)*(batch_size), 27643, score))
    
    if score > best_score:
        print('bestscore(C=%f): %f' %(C, score))
        best_score = score
        candidates.append({'clf' : clf, 'C' : C, 'score' : score})
"""

(27643, 439)
MNB score : 0.504793
SGD score : 0.506855
SGD score : 0.506855
svc score : 0.571284
lr score : 0.571284


"\nfor C in Cs:\n    alpha = C / 1000.0\n    clf = SGDClassifier(loss='hinge', alpha=alpha)\n    #stream = get_stream(path='train.csv', size=batch_size)\n    for i in range(iters):\n        #batch = next(stream)\n        #X_train, y_train = batch['Page content'], batch['Popularity']\n        start = (i)*batch_size\n        end = start + batch_size\n        if end >= X_train.shape[0] : end = X_train.shape[0]\n        \n        batch_X = X_train[start:end]\n        batch_y = y_train[start:end]\n        if X_train is None:\n            break\n        #X_train = sc_x.transform(tfidf.transform(X_train))\n        clf.partial_fit(batch_X, batch_y, classes=classes)\n    \n        score = clf.score(batch_X, batch_y)\n        print('[{}/{}] {}'.format((i+1)*(batch_size), 27643, score))\n    \n    if score > best_score:\n        print('bestscore(C=%f): %f' %(C, score))\n        best_score = score\n        candidates.append({'clf' : clf, 'C' : C, 'score' : score})\n"

In [77]:
print(selector.get_support())
c = 0
for i in selector.get_support():
    if i == True: c += 1
print(c)

[False False False ...,  True  True  True]
439


In [143]:
t0 = time()
X_test = tfidf.transform(df_test['Page content'])
duration = time() - t0

print('Finishing fitting test TFIDF : %f s' %(duration))
#X_test = sc_x.transform(X_test)

Finishing fitting test TFIDF : 61.200763 s


In [129]:
t0 = time()
X_test = hashvec.transform(df_test['Page content'])
duration = time() - t0
print('Finishing fitting test Hashvec : %f s' %(duration))


Finishing fitting test Hashvec : 64.255826 s


In [193]:
test_weekday = pd.read_csv('test_weekday.csv').values
test_time = pd.read_csv('test_time.csv').values

#X_test_con = hstack([X_test, test_weekday, test_time])
X_test_con = np.concatenate((test_weekday, test_time), axis=1)
#X_test_con = test_weekday * test_time
print(X_test_con.shape)
#X_test_std = sc_x.fit_transform(X_test_con)

(11847, 7)


In [194]:

for cand in candidates:
    predict = cand['clf'].predict(X_test_con)
    output = np.zeros((X_test_con.shape[0], 2), dtype=int)
    output[:, 0] = df_test['Id']
    output[:, 1] = predict
    df_output = pd.DataFrame(data=output, columns=['Id', 'Popularity'])
    df_output.to_csv('test%f.csv' % (cand['C']), index=False)